# Finding the vacancy formation energy of model aluminium

## Install lammmps software


In [ ]:
# Install lammps
!pip install lammps[mpi]
# Install package to assist in parsing lammps log files
!pip install lammps-logfile

## Mount your Google drive
...and change to the correct directory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
cd /content/drive/MyDrive/cdt_training/MaterialsModellingPrimer/03-VacancyFormation

## Import some useful modules 

In [3]:
import numpy as np
from scipy import optimize
import os
import shutil
import subprocess
from contextlib import chdir
import lammps_logfile
import matplotlib.pyplot as plt
%matplotlib inline

## Build and run simulations
(You will need to select the correct potential)

Creates and runs a series of simulations in increasingly large simulation cells.

In [ ]:
# Define a function to replace text in base input file
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

a = [4.0248454, 4.0452598, 4.0319604, 4.05, 4.050005, 4.0500102] # Equilibrium lattice parameter
ncells = [3,4,5,6,8]                            # Number of unit cells in each direction

pair_styles = ['eam/alloy', 'eam/fs', 'eam/alloy', 'eam/alloy', 'eam/alloy', 'eam/alloy']
potential_files = ['Pot1.set', 'Pot2.eam.fs', 'Pot3.eam.alloy', 'Pot4.eam.alloy', 'Pot5.eam.alloy', 'Pot6.eam.fs']
potential_index = 

if True:
    potential_file = potential_files[potential_index]
    pair_style = pair_styles[potential_index] 

    # Loop over number of unit cells in simulation
    # Create a folder and lammps input files for each variant
    for j in range(len(ncells)):
        # Create  a folder for simulation and copy in the empirical potential file
        path = './Simulations/' + 'potential_' + str(potential_index+1) +  '/' + 'Scale_'+ str(1.000) + '/' + 'Cells_' + str(ncells[j]) + '/'
        if not os.path.exists(path):
            os.makedirs(path, mode=0o777)
            shutil.copy2('../Potentials/' + potential_file, path)
        
        reference_atom = int((4*ncells[j]**3)/2)

        # Define values to replace in base input file
        replacements = {
            'ALATT':str(a[potential_index]),
            'NCELLS':str(ncells[j]),
            'DISP':str((ncells[j] + 0.5)*a[potential_index]/2.0),
            'POTENTIAL':potential_file,
            'PAIRSTYLE':pair_style,
            'REFATOM':str(reference_atom)
        }
        
        # Create a lammps file and use base file with string replacements to create unique variant
        inputLammpsFile = 'in_base.lmps'
        outputLammpsFile = path+'in.lmps'        
        finLammps = open(inputLammpsFile, 'r').read()
        foutLammps = open(outputLammpsFile, 'w')
        out = replace_all(finLammps, replacements)
        foutLammps.write(out)
        foutLammps.close()

        lammps_executable = 'lmp'
        lammps_command = '-in in.lmps'

        with chdir(path):
            os.system(lammps_executable + ' ' + lammps_command)

## Gather results
Gather the simulation results and write them to a file

In [5]:
if True:
    potential_file = potential_files[potential_index]

    output_path = './Simulations/' + 'potential_' + str(potential_index+1) + '/'

    with chdir(output_path):

        output_file = 'results.txt'
        colwidth = 24
        fo = open(output_file,'w')
        fo.write('#NCells'.ljust(colwidth) + 'Number of atoms'.ljust(colwidth) + 'Volume (A^3)'.ljust(colwidth) + 'Energy (eV)'.ljust(colwidth) + 'Pressure (GPa)'.ljust(colwidth) + '\n')

        for j in range(len(ncells)):
            # Create  a folder for simulation and copy in the empirical potential file
            path = 'Scale_'+ str(1.000) + '/' + 'Cells_' + str(ncells[j]) + '/'
            log = lammps_logfile.File(path + 'log.lammps')

            step = log.get("Step")
            pe = log.get("TotEng")
            length = log.get("Lx")
            pressure = log.get("Pxx")
            atoms = log.get("Atoms")


            fo.write(
                str(ncells[j]).ljust(colwidth) + 
                str(atoms[-1]).ljust(colwidth) + 
                str(length[-1]**3).ljust(colwidth) + 
                str(pe[-1]).ljust(colwidth) + 
                str(pressure[-1]/10000.0).ljust(colwidth) + 
                '\n'
                )
        fo.close()

## Load in the simulation results
Read the results of our simulations into a numpy array. Numpy makes this incredibly easy: one line is enough.

We then plot the formation energy as a function of the simulation cell size.

In [ ]:
fo = open('Simulations/results.txt', 'w')
fo.write('Potential'.ljust(colwidth) + 'Number of atoms'.ljust(colwidth) + 'Vacancy Energy (eV)'.ljust(colwidth) + 'Pressure (GPa)'.ljust(colwidth) + '\n')

output_path_bulk = '../01-LatticeParameters/Simulations/'
data_bulk = np.loadtxt(output_path_bulk + 'results.txt', skiprows=1)

numrows = 1
numcols = 1
fig,axes = plt.subplots(numrows, numcols, figsize=(4,3))

# Loop over potentials
if True:
    potential_file = potential_files[potential_index+1]
    output_path = './Simulations/' + 'potential_' + str(potential_index+1) + '/'
    data = np.loadtxt(output_path + 'results.txt', skiprows=1)

    E0 = data_bulk[potential_index,2]
    Ev = data[:,3] - data[:,1]*E0
    
    fo.write(str(potential_index).ljust(colwidth) + str(data[-1,1]).ljust(colwidth) + str(Ev[-1]).ljust(colwidth) + str(data[-1,4]).ljust(colwidth) + '\n')

    axes.plot(data[:,0],Ev[:], 'bo-')
    axes.set_xlabel('Number of cells')  
    axes.set_ylabel('Energy (eV)')
    axes.set_title(potential_file)
    fig.tight_layout()
fo.close()

print(f'Our estimate of the Vacancy formation energy is is {Ev[-1]:0.6f} eV')
